In [ ]:
# Pandas Core Methods — notebook-style runnable script
# ----------------------------------------------------
# How to run:
# 1) Put this in a file like `pandas_core_methods_demo.py` and run:
#       python pandas_core_methods_demo.py
#    OR paste cell-by-cell into a Jupyter notebook.
#
# 2) It will create:
#    - demo_data.csv
#    - demo_data.xlsx
#    - example.db (SQLite)
#    - output_plots/ (PNG charts you can use in a YouTube Short)

import os
import sqlite3
from pathlib import Path

import numpy as np
import pandas as pd


# -----------------------------
# Cell 1 — Setup output folders
# -----------------------------
OUT_DIR = Path("output_plots")
OUT_DIR.mkdir(exist_ok=True)




In [ ]:
# --------------------------------------------
# Cell 2 — Create a small demo dataset (source)
# --------------------------------------------
df_source = pd.DataFrame(
    {
        "id": [1, 2, 3, 4, 5, 6],
        "name": ["Ava", "Ben", "Ava", None, "Cara", "Ben"],
        "age": [23, 35, 23, 40, 29, 35],
        "city": ["London", "Paris", "London", "Berlin", "London", "Paris"],
        "score": [10, np.nan, 10, 30, 25, np.nan],
        "date": pd.to_datetime(
            ["2025-12-01", "2025-12-02", "2025-12-03", "2025-12-03", "2025-12-04", "2025-12-05"]
        ),
    }
)

# Add one duplicate row for drop_duplicates() demo
df_source = pd.concat([df_source, df_source.iloc[[2]]], ignore_index=True)

print("df_source:")
print(df_source)




In [ ]:
# -------------------------------------------------
# Cell 3 — Write demo files for read_csv/read_excel
# -------------------------------------------------
csv_path = Path("demo_data.csv")
xlsx_path = Path("demo_data.xlsx")

df_source.to_csv(csv_path, index=False)
df_source.to_excel(xlsx_path, index=False, sheet_name="Sheet1")

print(f"\nWrote: {csv_path.resolve()}")
print(f"Wrote: {xlsx_path.resolve()}")



In [ ]:

# -----------------------------------------
# Cell 4 — Create SQLite DB for read_sql demo
# -----------------------------------------
db_path = Path("example.db")
conn = sqlite3.connect(db_path)
df_source.to_sql("customers", conn, if_exists="replace", index=False)
conn.commit()
conn.close()
print(f"Wrote: {db_path.resolve()}")



In [ ]:

# ------------------------------
# Cell 5 — Load Data (I/O methods)
# ------------------------------
df_csv = pd.read_csv(csv_path, parse_dates=["date"])
df_xlsx = pd.read_excel(xlsx_path, sheet_name="Sheet1")
conn = sqlite3.connect(db_path)
df_sql = pd.read_sql("SELECT * FROM customers", con=conn)
conn.close()

print("\nLoaded df_csv (read_csv):", df_csv.shape)
print("Loaded df_xlsx (read_excel):", df_xlsx.shape)
print("Loaded df_sql (read_sql):", df_sql.shape)

# We'll use df_csv going forward
df = df_csv.copy()



In [ ]:

# ----------------
# Cell 6 — Inspect
# ----------------
print("\nhead():")
print(df.head(3))

print("\ninfo():")
df.info()

print("\ndescribe():")
print(df.select_dtypes(include=np.number).describe())

print("\ndtypes:")
print(df.dtypes)



In [ ]:

# -----------------------
# Cell 7 — Clean & Prepare
# -----------------------
print("\nMissing values (isna) by column:")
print(df.isna().sum())

# fillna()
df["score_filled"] = df["score"].fillna(0)

# dropna()
df_dropna_any = df.dropna()                 # drop rows with any NA
df_dropna_name = df.dropna(subset=["name"]) # drop rows where name is NA

# astype()
# (age is already int in many cases; ensure consistent)
df["age"] = df["age"].astype("int64")
df["city"] = df["city"].astype("category")

# drop_duplicates()
df_nodup_all = df.drop_duplicates()
df_nodup_subset = df.drop_duplicates(subset=["name", "age", "city", "score"])

print("\nAfter fillna(): example score_filled column")
print(df[["score", "score_filled"]].head())

print("\ndropna() results:")
print("df_dropna_any.shape =", df_dropna_any.shape)
print("df_dropna_name.shape =", df_dropna_name.shape)

print("\ndrop_duplicates() results:")
print("Original shape      =", df.shape)
print("No dups (all cols)  =", df_nodup_all.shape)
print("No dups (subset)    =", df_nodup_subset.shape)



In [ ]:

# -------------------------
# Cell 8 — Select & Filter
# -------------------------
# loc / iloc
first_name_loc = df.loc[0, "name"]
first_cell_iloc = df.iloc[0, 0]

london_rows_loc = df.loc[df["city"] == "London", ["name", "score", "date"]]
first2_rows_first3_cols = df.iloc[:2, :3]

# query()
age_30_plus = df.query("age >= 30")
london_and_score = df.query("city == 'London' and score_filled >= 10")

# isin()
city_in = df[df["city"].isin(["London", "Berlin"])]

print("\nloc example (df.loc[0, 'name']):", first_name_loc)
print("iloc example (df.iloc[0, 0]):", first_cell_iloc)

print("\nloc filter example (London):")
print(london_rows_loc)

print("\niloc slice example (first2 rows, first3 cols):")
print(first2_rows_first3_cols)

print("\nquery('age >= 30'):")
print(age_30_plus[["name", "age", "city", "score"]])

print("\nquery(\"city == 'London' and score_filled >= 10\"):")
print(london_and_score[["name", "age", "city", "score_filled"]])

print("\nisin(['London','Berlin']):")
print(city_in[["name", "city", "score"]])



In [ ]:

# ----------------
# Cell 9 — Transform
# ----------------
# assign()
df_assigned = df.assign(score2=df["score_filled"] * 2)

# rename()
df_renamed = df.rename(columns={"score": "points"})

# apply()
df["name_upper"] = df["name"].fillna("").apply(str.upper)
df["label"] = df.apply(lambda r: f"{r['city']}-{r['age']}", axis=1)

print("\nassign(): added score2")
print(df_assigned[["score_filled", "score2"]].head())

print("\nrename(): 'score' -> 'points'")
print(df_renamed.columns.tolist())

print("\napply(): name_upper + label")
print(df[["name", "name_upper", "label"]].head())



In [ ]:

# ----------------
# Cell 10 — Analyze
# ----------------
# groupby()
g = df.groupby("city")

# agg()
summary = df.groupby("city").agg(
    avg_age=("age", "mean"),
    avg_score=("score_filled", "mean"),
    n=("id", "count"),
)

# value_counts()
city_counts = df["city"].value_counts()
city_props = df["city"].value_counts(normalize=True)

# mean()
mean_age = df["age"].mean()
mean_numeric = df[["age", "score_filled"]].mean(numeric_only=True)

print("\nagg() summary by city:")
print(summary)

print("\nvalue_counts() city:")
print(city_counts)

print("\nvalue_counts(normalize=True) city:")
print(city_props)

print("\nmean() age:", mean_age)
print("\nmean() numeric columns:")
print(mean_numeric)



In [ ]:

# --------------------
# Cell 11 — Combine Data
# --------------------
left = pd.DataFrame({"id": [1, 2, 3, 7], "dept": ["Eng", "Sales", "Eng", "Ops"]})
right = pd.DataFrame({"id": [2, 3, 4], "bonus": [500, 900, 700]})

# merge()
merged = left.merge(right, on="id", how="left")

# concat()
df_a = pd.DataFrame({"id": [100, 101], "city": ["Rome", "Madrid"]})
df_b = pd.DataFrame({"id": [102, 103], "city": ["Lisbon", "Oslo"]})
stacked_rows = pd.concat([df_a, df_b], ignore_index=True)

print("\nmerge() result:")
print(merged)

print("\nconcat() result:")
print(stacked_rows)



In [ ]:

# ----------------
# Cell 12 — Visualize
# ----------------
# NOTE: Using pandas built-in plot methods (matplotlib backend).
# We save the figures to PNG so you can animate them in your YouTube Short.

ax1 = df["age"].plot(title="Age (line)")
fig1 = ax1.get_figure()
fig1.tight_layout()
fig1.savefig(OUT_DIR / "01_age_line.png")
fig1.clf()

ax2 = df["age"].plot.hist(bins=6, title="Age (hist)")
fig2 = ax2.get_figure()
fig2.tight_layout()
fig2.savefig(OUT_DIR / "02_age_hist.png")
fig2.clf()

ax3 = df[["age", "score_filled"]].plot.box(title="Age & Score (box)")
fig3 = ax3.get_figure()
fig3.tight_layout()
fig3.savefig(OUT_DIR / "03_age_score_box.png")
fig3.clf()

print(f"\nSaved plots to: {OUT_DIR.resolve()}")
print(" - 01_age_line.png")
print(" - 02_age_hist.png")
print(" - 03_age_score_box.png")



In [ ]:

# ----------------
# Cell 13 — Done ✅
# ----------------
print("\nDone. You now have a complete end-to-end demo for each method in the infographic.")
